In [ ]:
using Printf
using Plots
using FFTW

In [ ]:
# reverse a signal about the first element
flip(x) = x[-(0:length(x)-1) .& (length(x)-1) .+ 1]
swap(x) = conj(x)*im

odds(x) = x[1:2:length(x)]
evens(x) = x[2:2:length(x)]

linphase(offs, n) = cispi.(offs*-2(0:n-1)/n)
shift(x, offs) = x.*linphase(offs, length(x))

irev(i, N) = (-(i - 1) & (N - 1)) + 1

In [ ]:
N = 8
x, y = rand(0:9, N) + rand(0:9, N)im, rand(0:9, N) + rand(0:9, N)im
X, Y = fft(x), fft(y)
0

# FFT Identities

$x$ is a signal with elements from $\{x_n\}, n \in (1, N]$

$F(x) = X$

## Linearity

Addition and scalar multiplication work as expected.

$F(a \cdot x + b \cdot y) = a \cdot F(x) + b \cdot F(y)$

In [ ]:
round.([fft(2x + 3y) 2*fft(x) + 3*fft(y)]; digits=2)

## Flipping Time and Frequency

Flipping a signal around the first element will be denoted as $x^!$. Ex:

$x = {a, b, c, d}$

$x^! = {a, d, c, b}$

Flipping time flips frequency.

$F(x)^! = F(x^!)$

In [ ]:
round.([(flip∘fft)(x) (fft∘flip)(x)]; digits=2)

## Flipping via Conjugation

Conjugating the signal and spectrum is equal to flipping.

$F(x^*)^* = F(x^!) = F(x)^!$

In [ ]:
round.([(conj∘fft∘conj)(x) (flip∘fft)(x) (fft∘flip)(x)]; digits=2)

## Flipping via Swapping

Swapping the real/imaginary parts of the signal and spectrum is equal to flipping. This is useful if you have separate arrays for real/imag.

$F(x^* \cdot i)^* \cdot i = F(x^!) = F(x)^!$

In [ ]:
round.([(swap∘fft∘swap)(x) (flip∘fft)(x) (fft∘flip)(x)]; digits=2)

## Conjugate in Time

Conjugate in time is equal to the conjugate of the reverse in freq.

$F(x^*) = F(x^!)^* = F(x)^{!*} = F(x)^{*!}$

In [ ]:
round.([(fft∘conj)(x) (conj∘fft∘flip)(x) (conj∘flip∘fft)(x) (flip∘conj∘fft)(x)]; digits=2)

## Conjugate in Freq

Conjugate in freq is equal to the conjugate of the reverse in time.

$F(x)^* = F(x^*)^! = F(x^{*!}) = F(x^{!*})$

<!-- Swapping re/im acts similar to conjugation. (Note: I feel like there was a better identity to use here...)

`conj(fft(x)*im) = flip(conj(fft((x)))*im)` -->

In [ ]:
round.([(conj∘fft)(x) (flip∘fft∘conj)(x) (fft∘flip∘conj)(x) (fft∘conj∘flip)(x)]; digits=2)

## Real Part in Time/Freq

Real and imaginary parts in one domain have symmetries that can be exploited in the other. Linear transform!

$F(real(x)) = \frac{X + X^!*}{2}$

$F(real(x)) = \frac{X - X^!*}{2i}$

$F^{-1}(imag(X)) = \frac{x + x^!*}{2}$

$F^{-1}(imag(X)) = \frac{x - x^!*}{2i}$

In [ ]:
[fft(real(x)) (X + conj(flip(X)))/2]

## Almost it's own inverse

Same as FFT, but flipped and scaled.

$F^{-1}(x) \cdot N = F(x^!) = F(x)^! = F(x^*)^* = F(x^* \cdot i)^* \cdot i$

Always scaling the result by $\sqrt{N}$ is useful to avoid the iFFT scaling issue.

In [ ]:
round.([(ifft)(x)*N (fft∘flip)(x) (flip∘fft)(x) (conj∘fft∘conj)(x) (swap∘fft∘swap)(x)]; digits=2)

## Real/Imaginary Valued FFTs

Real valued FFTs have a symmetric result. ex:

$F(x) = [a, b, c, d, c^*, b^*, a^*]$

## Zero padding

Padding out a signal with zeros doesn't add any addtional information. It will increase the frequency resolution of the fft without adding any addition frequencies to it.

## Repetition

If you have a signal $x$, it will have an FFT of the form:

$F(x) = \{a, b, c, d\}$

If you repeat the signal, the even values will all be zeros. This is because the two halves are the same except for the phase. The even values are rotated by 180° so they cancel out.

$F(\{x, x\}) = 2\{a, 0, b, 0, c, 0, d, 0\}$

If $x$ is real, then you get the usual symmetry as well:

$F(\{x, x\}) = 2\{a, 0, b, 0, c, 0, b^*, 0\}$

## Other Sequences

Reversing a signal will be denoted as $x^@$. Ex:

$x = {a, b, c, d}$

$x^@ = {d, c, b, a}$

$F(\{x, x^@\}) = \{e, f, g, h, 0, -h, -g, -f\}$

In [ ]:
f() = fft(reverse(x))
g() = reverse(fft(x))
round.(f() - g(); digits = 10)

plot(
    plot([real(f()), real(g())]),
    plot([imag(f()), imag(g())]),
    plot([abs.(f()), abs.(g())]),
    label = ["f" "g"]
)

In [ ]:
plot()

xx = [x; reverse(x)]

fft((xx)).*linphase(0.5, 2N)

In [ ]:
z = zeros(ComplexF64, 2N)
z[2:2:2N] .= real(x)
Z = fft(z)

plot(
    plot(real(Z)),
    plot(imag(Z)),
    plot(abs.(Z)),
    label = ["f" "g"]
)
round.(Z; digits=3)

Z[0N + 1:1N] + Z[1N + 1:2N]
# [x; -x] sequence?

# Calculating a complex valued FFT

In [ ]:
# calculate FFT of odd elements
xo = Array(x)
xo[2:2:N] .= 0
Xo = fft(xo)

# calculate FFT of even elements
xe = Array(x)
xe[1:2:N] .= 0
Xe = fft(xe)

# FFT is linear, you can add the results
round.([Xo Xe Xo + Xe]; digits=2)

The FFT of a sequence with even elements 0s is the FFT of the odd elements repeated twice.

The FFT of a sequence with odd elements 0s is the FFT of the evens phase shifted by half a sample and repeated twice with the second repetition negated.

In [ ]:
# fft of evens and odds without zeros inserted
Xo = fft(x[1:2:N])
Xe = fft(x[2:2:N]).*linphase(0.5, N÷2)

# add them to get the full fft
round.([Xo Xe Xo + Xe; Xo -Xe Xo - Xe;]; digits=2)

# Real and Imaginary Valued FFTs

In [ ]:
# calculate FFT of just real values
Xre = fft(real(x))
Xim = fft(imag(x)im)

# FFT is linear, you can add the results!
round.([Xre Xim Xre + Xim]; digits=2)

In [ ]:
# with symmetry, you can extract them from the FFT as well
Xre = (X + conj(flip(X)))/2
Xim = (X - conj(flip(X)))/2
round.([Xre Xim]; digits=2)

# Real Valued FFTs

In [ ]:
N = 16
# xr = Array(1:N)
xr = rand(-9:9, N)
Xr = fft(xr)
round.([xr Xr]; digits=2)

In [ ]:
# map to a signal odds + evens*im
x2 = xr[1:2:N] + xr[2:2:N]im
p = fft(x2)/2
q = conj(flip(p))
# extract the evens/odds from the real/imag parts of the FFT
# apply final radix 2 Cooley/Tukey pass
w = -im.*linphase(0.5, N÷2)
Xo = (p + q)
Xe = (p - q).*w
X3 = [Xo + Xe; Xo[1] - Xe[1]]

round.(X3; digits=2)

## Real Valued iFFTs

In [ ]:
# extract the ffts of the real/imag
X4 = X3[1:N÷2]
_Xre = X4 - conj(flip(X4))
_Xre[1] = X3[1] - X3[N÷2+1]
_Xim = X4 + conj(flip(X4))
_Xim[1] = X3[1] + X3[N÷2+1]
# round.([Xre - _Xre Xim - _Xim]; digits = 2)

w = -im.*linphase(-0.5, N÷2)
_p = (_Xim - _Xre.*w)
# round.([p _p]; digits = 2)
# _X2 = ifft(_p)/2
_x = fft(flip(_p)/N)
round.(_X2; digits = 2)

[real(_x) imag(_x)]'[:]

# Calculating DCTs via the FFT

## DCT-II

In [ ]:
dct2(x) = real(fft([x; reverse(x)])[1:N].*linphase(0.25, N))
dct2(xr)

In [ ]:
x2 = [xr; reverse(xr)]
# x2 = [1:1:N; N:-1:1]

x2o = x2[1:2:2N]
# x2e = x2[2:2:2N]
# x2e = reverse(x2o) # x2e is also just the reverse of x2o

X2o = fft(x2o)
# X2e = fft(x2e)
# X2e = flip(X2o).*linphase(-1, N) # X2o can be flipped and phased to calculate X2e

# direct computation via evens/odds and a phase shift
# X2 = (X2o + X2e.*linphase(0.5, N)).*linphase(0.25, N)
# but X2e can be substituted out
# X2 = (X2o + flip(X2o).*linphase(-0.5, N)).*linphase(0.25, N)
# simplified by multiplying out the phases
X2 = X2o.*linphase(0.25, N) + flip(X2o).*conj(linphase(0.25, N))

# round.(y2; digits=2)
round.(X2 - dct2(xr); digits=2)

In [ ]:
x2 = [xr; reverse(xr)]
x2o = x2[1:2:2N]

# calculate X2o efficiently using a real valued FFT
p = fft(x2o[1:2:N] + x2o[2:2:N]*im)/2
q = conj(flip(p))
w = -im.*linphase(0.5, N÷2)
X2oo = (p + q)
X2oe = (p - q).*w
X2o = [X2oo + X2oe; X2oo - X2oe]

X2 = X2o.*linphase(0.25, N) + flip(X2o).*conj(linphase(0.25, N))
round.(X2 - dct2(xr); digits=2)

## DCT-III

In [ ]:
dct3(x) = real(fft([x; 0; -reverse(x); -x[2:N]; 0; reverse(x[2:N])])[2:2:2N])/4N
round.(dct3(dct2(xr)) - xr; digits=2)

In [ ]:
_X2 = dct2(xr)
# round.([X2 _X2]; digits=2)

w = linphase(-0.25, N)
_X2o = (_X2.*w - flip(_X2).*im.*w)/2
_X2o[1] = _X2[1]/2 # special case
round.([X2o _X2o]; digits=2)

# _x2o = fft(flip(_X2o))/N
_x2o = ifft(_X2o)
round.([xr x2o _x2o])

# deinterleve the parts
_xr = [_x2o[1:N÷2] _x2o[N:-1:N÷2+1]]'[:]
round.([dct3(dct2(xr)) .- _xr;])

# this version is wasteful, it calculates a complex FFT on a real valued signal

In [ ]:
_X2 = dct2(xr)

wm0, wm1 = cispi(-2/N), cispi(-0.5/N)
w0, w1 = wm0, wm1/2N

tmp = zeros(ComplexF64, N÷2)
tmp[1] = (X3[1]*(1 + im) + X3[N÷2+1]*(-1 + im))/N

for i0 in 2:N÷2
    i1 = irev(i0, N)
    p = (_X2[i0      ] + _X2[i1      ]*im)*w1
    q = (_X2[i1 - N÷2] - _X2[i0 + N÷2]*im)*w1*cispi(0.25)
    tmp[i0] = ((p - q)*w0 + (p + q)*im)
    w0 *= wm0; w1 *= wm1
end

_x2 = fft(tmp)
_x2o = [imag(_x2) real(_x2)]'[:]
_xr = [_x2o[1:N÷2] _x2o[N:-1:N÷2+1]]'[:]

round.([xr .- _xr;]; digits=3)

In [ ]:
# The packing order in the above looks like this...
foo = Array((1:N÷2) .- (1:N÷2)*im)
bar = [real(foo) imag(foo)]'[:]
baz = [bar[1:N÷2] bar[N:-1:N÷2+1]]'[:]

# looks like:
# [first_re last_im first_im last_im] then increment, etc

## DCT-IV

In [ ]:
dct4(x) = real(shift(fft([x; -reverse(x); -x; reverse(x)]), 0.5)[2:2:2N])

round.(dct4(xr); digits=2)
# norm4 = sqrt(2)/N
# dct4(dct4(xr))*norm4^2 - 2xr